# Predicting Prices for Houses in King County, WA

**Authors: Jesse Markowitz, Dillon Medd, Ronak Parekh**

## Overview

[overview text]

### Business problems/questions

Analysis and modeling through regression yield the following factors as predictive of house prices in King County:
 - Factor 1
 - Factor 2
 - Factor 3???

Together, these factors...

## Data Understanding

*Describe our data source, import libraries*

### Visualizations

*Put all our amazing viz: histograms, correlation heatmaps, the works. Really knock their socks off with this stuff.*

Explain why we log transform the stuff we do (prices, sqft_living) and show before/after histograms to show how it makes the distributions more normal looking. Because no one like weird-looking distributions. Get outta here with that skew.

### Feature engineering

*Here's where we explain, show, and execute our creation of features such as:*
 - Distance bins!
 - ???

## Modeling

*Show our modeling process and interpret the results!*

### Model validation

*Use R^2, rmse, QQ plots, etc. to explain why our model is the bomb dot com.*

## Conclusions

Hit 'em with that knowledge, son!

**Next steps:**
 - Future 1
 - Future 2
 - Future 3